<center>
    <h1>Football field Analysis</h1>
    <b>VIC Project, Main Code</b><br>
    <i>Coded By Chloé DAEMS, Amir MAHMOUDI & Anne-Claire LAISNEY</i>
</center>

<b> Sources :</b> <br>
<i>https://link.springer.com/chapter/10.1007/978-3-540-30125-7_101</i> pp 818-824. 2004 - Use hue detection to detect grass (ie field) -- The results seems not usable for us <br>
<i> https://static1.squarespace.com/static/5b048119f2e6b103db959419/t/5e99aeb4d85a234bb8752f78/1587130062444/Learning+to+track+and+identify+players+from+broadcast+sports+videos.pdf </i> Canny Edge detector to detect the field lines <br>
<i>https://www.cse.ust.hk/~quan/comp5421/notes/canny1986.pdf</i> Paper on the Canny detector

<b>Python libraries imports</b>

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import cv2
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

**Import the dataset of images**

In [2]:
import os
import pickle
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from skimage.io import imread
from random import shuffle
import scipy.io

def get_image_paths(data_path, fmt='jpg'):
    """
    This function returns lists containing the file path for each train 
    image, as well as lists with the label of each train image.
    :param data_path: path to the 'train' directories
    :param fmt: file extension of the images
    :return: lists: train_image_paths, train_labels
    """
    train_image_paths = []
    train_labels = []

    # train
    pth = os.path.join(data_path, '*.{:s}'.format(fmt))
    pth = glob(pth)
    shuffle(pth)
    train_image_paths.extend(pth)

    return train_image_paths

mat_1 = scipy.io.loadmat('annotation_1.mat')
mat_1 = mat_1['annot']
data_path = os.path.join('.', 'train')
train_image_paths = get_image_paths(data_path)
train_labels = mat_1

In [3]:
import scipy.io
import pandas as pd
# Set up parameters, image paths

df_ground_truth = pd.DataFrame(np.concatenate(mat_1))

for index, row in df_ground_truth.iterrows():
    row['ImgName'] = './train/'+row['ImgName'][0]

In [4]:
df_ground_truth

,BBox,ImgName
0,"[[86.0, 457.0, 116.0, 525.0], [148.0, 216.0, 1...",./train/0186.jpg
1,"[[86, 456, 115, 524], [148, 216, 163, 258], [2...",./train/0187.jpg
2,"[[81.0, 450.0, 120.0, 532.0], [146.0, 216.0, 1...",./train/0188.jpg
3,"[[89.0, 455.0, 115.0, 523.0], [147.0, 218.9999...",./train/0189.jpg
4,"[[90.0, 455.0, 114.0, 524.0], [148.0, 220.0000...",./train/0190.jpg
...,...,...
1490,"[[473.0, 414.99999999999994, 496.0, 481.999999...",./train/2095.jpg
1491,"[[1199.0, 516.0, 1223.0, 585.0], [1050.0, 410....",./train/2096.jpg
1492,"[[471.0, 413.0, 493.0, 481.0], [383.0, 226.0, ...",./train/2097.jpg
1493,"[[1196.0, 515.0, 1221.0, 585.0], [1043.0, 410....",./train/2098.jpg


In [5]:
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from skimage.io import imread
from skimage.transform import resize

H, W = 720, 1280
N = len(df_ground_truth)

def read_frame(train_image_paths, frame):
    file_path = train_image_paths[frame]
    img = cv2.imread(file_path,cv2.IMREAD_GRAYSCALE)
    img = resize(img, output_shape=(H,W))
    return img

def annotations_for_frame(df_annotation, frame):
    assert frame in df_annotation.index
    bbs = df_annotation[df_annotation.index == frame].values[0]
    return bbs

def show_annotation(df_annotation, frame):
    img = read_frame(df_annotation['ImgName'], frame)
    bbs = annotations_for_frame(df_annotation['BBox'], frame)
    fig, ax = plt.subplots(figsize=(20, 12))
    print(type(ax))
    for x1, y1, x2, y2 in bbs:
        dx = x2 - x1
        dy = y2 - y1
        rect = patches.Rectangle((x1, y1), dx, dy, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    ax.imshow(img)
    ax.set_title('Annotations for frame {}.'.format(frame))

def bounding_boxes_to_mask(bounding_boxes, H, W):
    
    """
    Converts set of bounding boxes to a binary mask
    """
    mask = np.zeros((H, W))
    for x1, y1, x2, y2 in bounding_boxes:
        dx = x2 - x1
        dy = y2 - y1
        mask[y1:y2,x1:x2] = 1
    return mask

def run_length_encoding(mask):

    """
    Produces run length encoding for a given binary mask
    """
    
    # find mask non-zeros in flattened representation
    non_zeros = np.nonzero(mask.flatten())[0]
    padded = np.pad(non_zeros, pad_width=1, mode='edge')
    
    # find start and end points of non-zeros runs
    limits = (padded[1:] - padded[:-1]) != 1
    starts = non_zeros[limits[:-1]]
    ends = non_zeros[limits[1:]]
    lengths = ends - starts + 1
    
    return ' '.join(['%d %d' % (s, l) for s, l in zip(starts, lengths)])

In [7]:
from ipywidgets import interact, widgets
from IPython.display import display

def f_display_frame(frame_id):
    show_annotation(df_ground_truth,frame_id)

interact(f_display_frame, frame_id=widgets.IntSlider(min=1, max=1493, step=1, value=1))

interactive(children=(IntSlider(value=1, description='frame_id', max=1493, min=1), Output()), _dom_classes=('w…

<function __main__.f_display_frame(frame_id)>

## Step 1 : Canny Edge detector

**We are first going to try a step by step pipeline for image preparation and the canny edge detector, inspired by the work done in assignment 1 part 2.**

**A. Smooth the image**

In [8]:
from scipy.ndimage import gaussian_filter
smoothed = []

def show_smoothed(df_annotation, frame):
    img = read_frame(df_annotation['ImgName'], frame)
    smoothed_ = gaussian_filter(img, sigma=2)
    smoothed.append(smoothed_)
    fig, axes = plt.subplots(figsize=(20, 12), ncols=2)
    axes[0].imshow(img) 
    axes[1].imshow(smoothed_)
    
    bbs = annotations_for_frame(df_annotation['BBox'], frame)
    for x1, y1, x2, y2 in bbs:
        dx = x2 - x1
        dy = y2 - y1
        rect = patches.Rectangle((x1, y1), dx, dy, edgecolor='r', facecolor='none')
        axes[0].add_patch(rect)

In [9]:
def f_display_smoothed(frame_id):
    show_smoothed(df_ground_truth,frame_id)

interact(f_display_smoothed, frame_id=widgets.IntSlider(min=1, max=853, step=1, value=1))

interactive(children=(IntSlider(value=1, description='frame_id', max=853, min=1), Output()), _dom_classes=('wi…

<function __main__.f_display_smoothed(frame_id)>

**B. Gradients**

In [10]:
from skimage.filters import sobel 

gradient_g_magnitude = []
gradient_g_dir = []

def gradient(img):
    g_x = sobel(img, axis=1) 
    g_y = sobel(img, axis=0)
    g_mag = np.sqrt(g_x**2 + g_y**2) 
    g_dir = np.arctan(g_y / g_x)
    return g_mag, g_dir

def show_gradient(df_annotation, frame):
    img = read_frame(df_annotation['ImgName'], frame)
    smoothed_ = gaussian_filter(img, sigma=2)
    g_magnitude, g_dir = gradient(smoothed_)
    gradient_g_magnitude.append(g_magnitude)
    gradient_g_dir.append(g_dir)
    fig, axes = plt.subplots(figsize=(20, 12), ncols=2)
    axes[0].imshow(g_magnitude) 
    axes[1].imshow(g_dir)
    
    bbs = annotations_for_frame(df_annotation['BBox'], frame)
    for x1, y1, x2, y2 in bbs:
        dx = x2 - x1
        dy = y2 - y1
        rect = patches.Rectangle((x1, y1), dx, dy, edgecolor='r', facecolor='none')
        axes[0].add_patch(rect)

In [11]:
def f_display_gradient(frame_id):
    show_gradient(df_ground_truth,frame_id)

interact(f_display_gradient, frame_id=widgets.IntSlider(min=1, max=853, step=1, value=1))

interactive(children=(IntSlider(value=1, description='frame_id', max=853, min=1), Output()), _dom_classes=('wi…

<function __main__.f_display_gradient(frame_id)>

**C. Non-maximum suppression**

In [12]:
def non_maximum_suppression(g_magnitude, g_dir):
    #quantise the gradient directions into 4 values np.array([0, np.pi/4, np.pi/2, 3*np.pi/4])
    h, w = g_magnitude.shape 
    values = np.array([0, np.pi, np.pi/4, np.pi/2, 3*np.pi/4])
    g_max = np.zeros_like(g_magnitude)
    for i in range(h-1):
        for j in range(w-1):
            diff = abs(values - g_dir[i,j])
            angle = values[np.argmin(diff)]
            q = 0
            r = 0
            #case angle = 0
            if angle == 0 or angle == 1 :
                q = g_magnitude[i, j+1]
                r = g_magnitude[i, j-1]
            #case angle = pi/4
            elif angle == 2:
                q = g_magnitude[i+1, j-1]
                r = g_magnitude[i-1, j+1]
            #case angle = pi/2 
            elif angle == 3:
                q = g_magnitude[i+1, j]
                r = g_magnitude[i-1, j]
            #case angle = 3pi/4
            else:
                q = g_magnitude[i-1, j-1]
                r = g_magnitude[i+1, j+1]
                
            if g_magnitude[i,j] >= q and g_magnitude[i,j]>=r : 
                g_max[i,j] = g_magnitude[i,j]
            else:
                g_max[i,j] = 0

    return g_max

In [13]:
def show_non_maximum_suppression(df_annotation, frame):
    img = read_frame(df_annotation['ImgName'], frame)
    smoothed_ = gaussian_filter(img, sigma=2)
    g_magnitude, g_dir = gradient(smoothed_)
    g_max = non_maximum_suppression(g_magnitude, g_dir)
    fig, axes = plt.subplots(figsize=(20, 12), ncols=2)
    axes[0].imshow(g_max, cmap='gray')
    axes[1].imshow(g_magnitude - g_max, cmap='gray')
    
    bbs = annotations_for_frame(df_annotation['BBox'], frame)
    for x1, y1, x2, y2 in bbs:
        dx = x2 - x1
        dy = y2 - y1
        rect = patches.Rectangle((x1, y1), dx, dy, edgecolor='r', facecolor='none')
        axes[0].add_patch(rect)
        
def f_display_non_maximum_suppression(frame_id):
    show_non_maximum_suppression(df_ground_truth,frame_id)
    
interact(f_display_non_maximum_suppression, frame_id=widgets.IntSlider(min=1, max=853, step=1, value=1))

interactive(children=(IntSlider(value=1, description='frame_id', max=853, min=1), Output()), _dom_classes=('wi…

<function __main__.f_display_non_maximum_suppression(frame_id)>

**D. Double Thresholding**

In [14]:
def double_thresholding(g_max, thresh_lo, thresh_hi):
    h, w = g_max.shape
    thresh_img = np.zeros_like(g_max)
    highThreshold = g_max.max() * thresh_hi;
    lowThreshold = highThreshold * thresh_lo;
    
    for i in range(h):
        for j in range(w):
            if g_max[i,j] > highThreshold :
                 thresh_img[i,j] = 1
            elif g_max[i,j] > lowThreshold:
                thresh_img[i,j] = 0.25

    return thresh_img

def show_double_thresholding(df_annotation, frame):
    img = read_frame(df_annotation['ImgName'], frame)
    smoothed_ = gaussian_filter(img, sigma=2)
    g_magnitude, g_dir = gradient(smoothed_)
    g_max = non_maximum_suppression(g_magnitude, g_dir)
    thresh_img = double_thresholding(g_max, thresh_lo=0.05, thresh_hi=0.08)
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(thresh_img)
    
    bbs = annotations_for_frame(df_annotation['BBox'], frame)
    for x1, y1, x2, y2 in bbs:
        dx = x2 - x1
        dy = y2 - y1
        rect = patches.Rectangle((x1, y1), dx, dy, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

In [66]:
def f_double_thresholding(frame_id):
    show_double_thresholding(df_ground_truth,frame_id)
    
interact(f_double_thresholding, frame_id=widgets.IntSlider(min=1, max=853, step=1, value=1))

interactive(children=(IntSlider(value=1, description='frame_id', max=853, min=1), Output()), _dom_classes=('wi…

<function __main__.f_double_thresholding(frame_id)>

**E. Edge connectivity**

In [15]:
def connectivity(thresh_img):
    h, w = thresh_img.shape
    edge_img = thresh_img
    
    for i in range(h):
        for j in range(w):
            if edge_img[i,j] == 0.25:
                if edge_img[i+1,j] == 1 or edge_img[i+1,j+1] == 1 or edge_img[i,j+1] == 1 or edge_img[i-1,j] == 1 or edge_img[i-1,j-1] == 1 or edge_img[i,j-1] == 1 or edge_img[i+1,j-1] == 1 or edge_img[i-1,j+1] == 1:
                    edge_img[i,j] = 1 
                else:
                    edge_img[i,j] = 0

    return edge_img

In [16]:
def show_connectivity(df_annotation, frame):
    img = read_frame(df_annotation['ImgName'], frame)
    smoothed_ = gaussian_filter(img, sigma=2)
    g_magnitude, g_dir = gradient(smoothed_)
    g_max = non_maximum_suppression(g_magnitude, g_dir)
    thresh_img = double_thresholding(g_max, thresh_lo=0.05, thresh_hi=0.08)
    edge_img = connectivity(thresh_img)
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(edge_img, cmap='gray')
    
    bbs = annotations_for_frame(df_annotation['BBox'], frame)
    for x1, y1, x2, y2 in bbs:
        dx = x2 - x1
        dy = y2 - y1
        rect = patches.Rectangle((x1, y1), dx, dy, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

In [17]:
def f_double_connectivity(frame_id):
    show_connectivity(df_ground_truth,frame_id)
    
interact(f_double_connectivity, frame_id=widgets.IntSlider(min=1, max=853, step=1, value=1))

interactive(children=(IntSlider(value=1, description='frame_id', max=853, min=1), Output()), _dom_classes=('wi…

<function __main__.f_double_connectivity(frame_id)>

#### Canny detector Pipeline

In [20]:
def canny_edge_detector(img, thresh_lo=0.1, thresh_hi=0.2):
    """
    The Canny edge detector.
    
    Inputs:
        img              The input image
        thresh_lo        The fraction of the maximum gradient magnitude which will 
                         be considered the lo threshold. 
        thresh_hi        The fraction of the maximum gradient magnitude which will
                         be considered the hi threshold. Ideally should be 2x to 3x 
                         thresh_lo.
                         
    Outputs: 
        edge_img         A binary image, with pixels lying on edges marked with a 1, 
                         and others with a 0.
    """
    
    # Smooth the image first. 
    smoothed             = gaussian_filter(img,2)
    
    # Find gradient magnitude and direction
    g_magnitude, g_dir   = gradient(smoothed)
    
    # Non-maximum suppression
    g_max                = non_maximum_suppression(g_magnitude, g_dir)
    
    # Double thresholding
    thresh_img           = double_thresholding(g_max, thresh_lo, thresh_hi)
    
    # Final edge connectivity
    edge_img             = connectivity(thresh_img)
    
    # Return the result
    return edge_img

def show_canny_edge_detector(df_annotation, frame):
    img = read_frame(df_annotation['ImgName'], frame)
    edges = canny_edge_detector(img, thresh_lo=0.05, thresh_hi=0.09)
    fig, axes = plt.subplots(figsize=(20, 12), ncols=2)
    axes[0].imshow(img, cmap='gray')
    axes[1].imshow(edges, cmap='gray')
    
    bbs = annotations_for_frame(df_annotation['BBox'], frame)
    for x1, y1, x2, y2 in bbs:
        dx = x2 - x1
        dy = y2 - y1
        rect = patches.Rectangle((x1, y1), dx, dy, edgecolor='r', facecolor='none')
        axes[0].add_patch(rect)

In [21]:
def f_canny_edge_detector(frame_id):
    show_canny_edge_detector(df_ground_truth,frame_id)
    
interact(f_canny_edge_detector, frame_id=widgets.IntSlider(min=1, max=853, step=1, value=1))

interactive(children=(IntSlider(value=1, description='frame_id', max=853, min=1), Output()), _dom_classes=('wi…

<function __main__.f_canny_edge_detector(frame_id)>

<div class="alert alert-block alert-success">
<b>Next step:</b> Here, you can see that the players are noising the detections of the fields'line, in the Lu's Paper they decided to detect the bounding boxes of the players to delete them from the edge detection. This is what we are going to try to do next.
</div>

### Find the players on the field
Main ref paper : http://cs.brown.edu/people/pfelzens/papers/latent.pdf (seen in course)

https://www.cs.toronto.edu/~fidler/slides/2015/CSC420/lecture19.pdf (course on DPM detector -- based on Hog detector)

http://ipl.ce.sharif.edu/azadi_soccer_dataset.html (dataset 1 -- emailed the owner)

**First we try a classic HOG + SVM from opencv**
The DPM model starts by borrowing the idea of the HOG detector

In [22]:
!pip3 install imutils

In [ ]:
from imutils.object_detection import non_max_suppression
import argparse
import imutils

# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

#image = imutils.resize(image, height = 350, width=625)
#image = resize(img, output_shape=(350, 625))
orig = image.copy()
# detect people in the image
(rects, weights) = hog.detectMultiScale(image, winStride=(5, 5), scale = 1.01)
# draw the original bounding boxes
for (x, y, w, h) in rects:
    cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)
# apply non-maxima suppression to the bounding boxes using a
# fairly large overlap threshold to try to maintain overlapping
# boxes that are still people
rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
pick = non_max_suppression(rects, probs=None, overlapThresh=0.9)
# draw the final bounding boxes
for (xA, yA, xB, yB) in pick:
    cv2.rectangle(image, (xA, yA), (xB, yB), (0, 255, 0), 2)
# show some information on the number of bounding boxes
print("[INFO]: {} original boxes, {} after suppression".format(
     len(rects), len(pick)))

# show the output images
fig, axes = plt.subplots(figsize=(20, 12), ncols=2)
axes[0].imshow(orig, cmap='gray')
axes[1].imshow(image, cmap='gray')


**Let's code a Hog detector**


In [ ]:
def sliding_window(image, window_size, step_size=8):
    # slide a window across the image
    for y in range(0, image.shape[0], step_size):
        for x in range(0, image.shape[1], step_size):
            # yield the current window
            window = image[y:y + window_size[1], x:x + window_size[0]]
            if not (window.shape[0] != window_size[1] or window.shape[1] != window_size[0]):
                yield (x, y, window)

In [ ]:
# load the image, convert it to grayscale, dand detect edges
winStride = (8, 8)  
padding = (8, 8)
locations = ((10,20),)

data = []   # Feature vectors
labels = [] # Label for each feature vector
(winW, winH) = (200, 100) # Window size for images
min_loss = 10.0 # Min loss so that a result is considered valid

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edged = imutils.auto_canny(gray)
contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
c = max(contours, key=cv2.contourArea)
H = hog.compute(image,winStride,padding,locations)

In [ ]:
data.append(H)